In [1]:
import tensorflow as tf
import numpy as np


In [2]:
class InputPipeline:
  def __init__(self,batch_size=64):
    self.batch_size = batch_size

  @staticmethod
  def load_dataset():
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    X_train = X_train.astype("float32")/255.0
    X_test = X_test.astype("float32")/255.0
    y_train = y_train.astype("int32")
    y_test = y_test.astype("int32")
    return X_train, y_train, X_test, y_test
  
  def data_generator(self,features,labels):
    dataset = tf.data.Dataset.from_tensor_slices((features,labels))
    dataset = dataset.batch(batch_size=self.batch_size,
                            drop_remainder=True)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

  def __call__(self):
    X_train, y_train, X_test, y_test = self.load_dataset()
    train_dataset = self.data_generator(X_train, y_train)
    test_dataset = self.data_generator(X_test, y_test)
    return train_dataset, test_dataset

In [3]:
class Model:
  def __init__(self,train_dataset,
               test_dataset,
               image_width=32,
               image_height=32,
               classes=10,
               epochs=1):
    self.train_dataset = train_dataset
    self.test_dataset = test_dataset
    self.image_width = image_width
    self.image_height = image_height 
    self.classes = classes
    self.epochs = epochs
  
  def mobilenet_model(self):
    base_model = tf.keras.applications.MobileNetV2(include_top=False,
                                                   input_shape=(self.image_width,
                                                                self.image_height,
                                                                3),
                                                  weights="imagenet")
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    out = tf.keras.layers.Dense(self.classes)(x)
    model = tf.keras.models.Model(base_model.input,out)
    return model
  
  def __call__(self):
    model = self.mobilenet_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-07,
                                                amsgrad=False),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=["accuracy"])
    model.summary()
    model.fit(self.train_dataset,epochs=self.epochs,
              validation_data=self.test_dataset)
    model.save("mobilenet.h5")
    score = model.evaluate(self.test_dataset,batch_size=64, verbose=0)
    print("Model acc: ", score[1])

In [4]:
def main_model():
  data = InputPipeline()
  X_train, y_train, X_test, y_test = data.load_dataset()
  train_dataset, test_dataset = data.__call__()

  model = Model(train_dataset=train_dataset,
                test_dataset=test_dataset)
  model.__call__()

if __name__ == "__main__":
  main_model()

9412608/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 16, 16, 32)   0           bn_Conv1[0][0]                   
_______________________________

In [8]:
!pip install tf2onnx onnx onnxruntime

['Collecting tf2onnx',
 '\x1b[?25l  Downloading https://files.pythonhosted.org/packages/db/32/33ce509a79c207a39cf04bfa3ec3353da15d1e6553a6ad912f117cc29130/tf2onnx-1.8.4-py3-none-any.whl (345kB)',
 '',
 '\x1b[K     |█                               | 10kB 24.5MB/s eta 0:00:01',
 '\x1b[K     |██                              | 20kB 30.1MB/s eta 0:00:01',
 '\x1b[K     |██▉                             | 30kB 21.3MB/s eta 0:00:01',
 '\x1b[K     |███▉                            | 40kB 25.0MB/s eta 0:00:01',
 '\x1b[K     |████▊                           | 51kB 21.6MB/s eta 0:00:01',
 '\x1b[K     |█████▊                          | 61kB 24.2MB/s eta 0:00:01',
 '\x1b[K     |██████▋                         | 71kB 19.4MB/s eta 0:00:01',
 '\x1b[K     |███████▋                        | 81kB 20.5MB/s eta 0:00:01',
 '\x1b[K     |████████▌                       | 92kB 19.2MB/s eta 0:00:01',
 '\x1b[K     |█████████▌                      | 102kB 19.6MB/s eta 0:00:01',
 '\x1b[K     |██████████▍             

In [9]:
tf.saved_model.save(model, "mobilenet")
!python -m tf2onnx.convert --saved-model mobilenet --output "mobilenet.onnx"

INFO:tensorflow:Assets written to: mobilenet/assets
2021-04-20 07:18:49.527795: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-04-20 07:18:50.848204: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-20 07:18:50.849053: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-20 07:18:50.854662: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-20 07:18:50.855123: I tensorflow/core/common_runtime/gpu/g

In [13]:
noise = tf.random.uniform((1, 32, 32 ,3))

import onnxruntime

onnx_session = onnxruntime.InferenceSession("mobilenet.onnx")
onnx_inputs = {onnx_session.get_inputs()[0].name: noise.numpy()}
onnx_output = onnx_session.run(None, onnx_inputs)[0]

print("Predicted class: ",np.argmax(onnx_output))

Predicted class:  2
